In [1]:
                                                import json, os
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
morph = MorphAnalyzer()
stops = set(stopwords.words('russian'))

In [2]:
#import nltk
#nltk.download()

In [2]:
pd.set_option('display.max_colwidth', 1000)

## Данные

In [3]:
# скачаем данные в папке data и распакуем их
PATH_TO_DATA = './data' #ng_0.jsonlines
PATH_TO_DATA = './ng'

In [4]:
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA)]

Объединим файлы в один датасет.

In [5]:
data = pd.concat([pd.read_json(file, lines=True) for file in files][:1], axis=0, ignore_index=True)

In [6]:
data.shape

(988, 5)

In [7]:
#data.head(6)

In [8]:
# за что отвечает [1:2], и почему мы берём меньше?
data1 = pd.concat([pd.read_json(file, lines=True) for file in files][1:2], axis=0, ignore_index=True)

In [38]:
def evaluate(true_kws, predicted_kws):
  #  assert len(true_kws) == len(predicted_kws)
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec+rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    print('Precision - ', round(np.mean(precisions), 2)) # 2 means ?
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))
    
    
        

Проверим, что всё работает как надо.

In [39]:
evaluate(data['keywords'], data['keywords'])

Precision -  1.0
Recall -  1.0
F1 -  1.0
Jaccard -  1.0


## Токенизация, удаление стоп-слов и нормализация.

In [14]:
from string import punctuation
from nltk.corpus import stopwords
punct = punctuation+'0123456789«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return words

In [15]:
import time
start = time.time()
data['content_norm'] = data['content'].apply(normalize)
end = time.time()
print(end - start)
    

179.81707334518433


In [37]:
def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS == 'NOUN']

    return words

In [38]:
data['content_norm'] = data['content'].apply(normalize)

In [35]:
evaluate(data['keywords'], data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(7)]))

Precision -  0.13
Recall -  0.19
F1 -  0.15
Jaccard -  0.09


# Способ №1.

Сделать min_df = 2 (means "ignore terms that appear in less than 2 documents”).
[Это ничего не изменит, но можно ещё сделать max_df = 0.80 (means "ignore terms that appear in more than 80% documents”)]
Результат: F1 = 0.17

In [58]:
# можно заодно сделать нграммы
tfidf = TfidfVectorizer(ngram_range=(1,3), max_df = 0.80, min_df=2)

In [47]:
tfidf.fit(data['content_norm_str'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.8, max_features=None, min_df=2,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [48]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}

Преобразуем наши тексты в векторы, где на позиции i стоит tfidf коэффициент слова i из словаря.

In [49]:
texts_vectors = tfidf.transform(data['content_norm_str'])

Отсортируем векторы текстов по этим коэффициентам и возьмем топ-10.

In [53]:
# сортировка по убыванию, поэтому нужно развернуть список
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-11:-1]] 

In [57]:
keywords[:2]

[['ольга васильев',
  'стандарт',
  'васильев',
  'ольга',
  'источник',
  'предмет',
  'письмо',
  'произведение',
  'редакция',
  'содержание'],
 ['красота',
  'отчаяние',
  'порыв',
  'кошка',
  'глаз',
  'руина',
  'красота глаз',
  'предмет',
  'банкнот',
  'занавеска']]

In [55]:
evaluate(data['keywords'], keywords)

Precision -  0.13
Recall -  0.25
F1 -  0.17
Jaccard -  0.1


F-мера стала лучше на 0.01, но всё ещё попадается мусор ("при", "ольга", "борода" - такие себе ключевые слова).

# Способ № 2. 


Если брать не 10 слов из обработанных, а 7, то получится ещё чуть лучше:
F1 -  0.18

In [59]:
# исходная версия tfidf, не как в способе № 1
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=5)

In [60]:
# сортировка по убыванию, поэтому нужно развернуть список
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-7:-1]] 

In [61]:
tfidf.fit(data['content_norm_str'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=5,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [62]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}

In [63]:
texts_vectors = tfidf.transform(data['content_norm_str'])

In [90]:
# сортировка по убыванию, поэтому нужно развернуть список
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-6:-1]] 

In [91]:
keywords[:2]

[['ольга васильев', 'стандарт', 'васильев', 'ольга', 'источник'],
 ['красота', 'отчаяние', 'порыв', 'кошка', 'глаз']]

In [92]:
evaluate(data['keywords'], keywords)

Precision -  0.19
Recall -  0.18
F1 -  0.17
Jaccard -  0.11


In [33]:
keywords

['элитарным развлечением',
 'экспрессивным слогом',
 'шарль дидло',
 'чудовищными ошибками',
 'художественный феномен',
 'феерия ?..',
 'театральных афишах',
 'театр вслед',
 'сумел закрепиться',
 'стоила одна',
 'современным балетам',
 'собирал знания',
 'словам яковлевой',
 'склоне дней',
 'сергей худеков',
 'самых истоков',
 'русскую почву',
 'русские танцовщицы',
 'разнообразные вопросы',
 'равно просачивались',
 'прозвищу веретено',
 'приклеивай бренд',
 'приказ короля',
 'правильно подавать',
 'пору конкурировали',
 'попытка увидеть',
 'петербургской газеты',
 'отмечает автор',
 'отзыву современника',
 'особым удовольствием',
 'обманывали ожиданий',
 'неоткрытой звезды',
 'невиданного совершенства',
 'начинает исследовательница',
 'намоченным свернутым',
 'надолго воцарились',
 'молниеносные прыжки',
 'модные спектакли',
 'многом умалчивают',
 'многие танцовщицы',
 'мире »).',
 'министру двора',
 'мариинском театре',
 'маневров гвардии',
 'ловким карьеристом',
 'красном селе',
 '

# Способ № 3

Если соединить способы 1 и 2, F-мера станет 0.18.

In [93]:
tfidf = TfidfVectorizer(ngram_range=(1,3), max_df = 0.80, min_df=2)

tfidf.fit(data['content_norm_str'])

id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}

texts_vectors = tfidf.transform(data['content_norm_str'])

keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-6:-1]] 

keywords[:2]

evaluate(data['keywords'], keywords)

Precision -  0.19
Recall -  0.18
F1 -  0.18
Jaccard -  0.11


# Способ № 4

Попробовать новые стоп-слова на просто частотной выборке. F1 -  0.17

In [106]:
from string import punctuation
from nltk.corpus import stopwords
from stop_words import get_stop_words

punct = punctuation+'«»—…“”*№–' # '0123456789«»—…“”*№–' изначально было без цифр
stop_words = set(get_stop_words('ru'))
#print(stop_words)

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stop_words]
    words = [word.normal_form for word in words if word.tag.POS == 'NOUN']

    return words

In [107]:
data['content_norm'] = data['content'].apply(normalize)

In [108]:
# топ 10 частотных слов статьи
evaluate(data['keywords'], data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.14
Recall -  0.26
F1 -  0.17
Jaccard -  0.1


А если использовать с новыми стоп-словами tf-idf, ничего не изменится.

In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1,3), max_df = 0.80, min_df=2)

tfidf.fit(data['content_norm_str'])

id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}

texts_vectors = tfidf.transform(data['content_norm_str'])

keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-6:-1]] 

keywords[:2]

evaluate(data['keywords'], keywords)

Возьмем этот результат за baseline. 

Precision -  0.13
Recall -  0.24
F1 -  0.16
Jaccard -  0.09

# Способ №5. 

Попробуем TermExtractor, F-мера станет 0.17

Источник: https://github.com/igor-shevchenko/rutermextract
nested true попробовать

In [50]:
from rutermextract import TermExtractor

term_extractor = TermExtractor()

vse_1 = []
for i in data['content']:
    t_extr = []
    for term in term_extractor(''.join(i), limit = 5):
        t_extr.append(term.normalized)
    vse_1.append(t_extr)

In [51]:
print(len(vse_1))

988


In [52]:
evaluate(data['keywords'], vse_1)

Precision -  0.18
Recall -  0.17
F1 -  0.17
Jaccard -  0.1


# Способ № 6

Попробуем Rake. Жалкая F-мера: 0.03.

Из NLTK: https://github.com/csurfer/rake-nltk



In [63]:
from rake_nltk import Rake

In [64]:
from stop_words import get_stop_words

punctuations = punctuation+'0123456789«»—…“.,”*№–' # '0123456789«»—…“”*№–' изначально было без цифр
stop_words = set(get_stop_words('ru'))

In [105]:
r = Rake(language="ru", stopwords=list(stop_words), punctuations=punctuations, min_length=1, max_length=1)

# Extraction given the text.
rake_keywords = []
for text in data['content']:
    r.extract_keywords_from_text(text)
    keywords = r.get_ranked_phrases()
    #c=r.get_ranked_phrases_with_scores()
    rake_keywords.append(keywords[:20])
print(rake_keywords)

[['явлениях', 'явления', 'явлений', 'читают', 'читали', 'читала', 'числе', 'цифра', 'фгосам', 'фгос', 'ученики', 'уроке', 'увязываю', 'темы', 'таблицу', 'схему', 'страничке', 'стихотворений', 'стихи', 'стать'], ['ясно', 'шах', 'шалея', 'чьих', 'чужая', 'чеширски', 'честью', 'храмина', 'хорошим', 'хорошие', 'хорошее', 'ходит', 'хлоп', 'уясните', 'ушла', 'утверждении', 'успокоительное', 'условия', 'умиротворение', 'укорот'], ['якутского', 'якутии', 'юзефович', 'экзотический', 'штаны', 'шинели', 'холода', 'х', 'фронты', 'фаталист', 'успех', 'унылой', 'тунгусов', 'точь', 'томске', 'той', 'темой', 'тема', 'сюжету', 'сысы'], ['является', 'эффективно', 'эндом', 'энд', 'штабе', 'чудо', 'чувствовал', 'чемпионате', 'целом', 'хромают', 'хочу', 'формулы', 'формуле', 'формула', 'финише', 'финиша', 'учитывать', 'уступали', 'умеют', 'уильямсы'], ['яростью', 'юпитеры', 'юбилейное', 'эрлихом', 'эрлих', 'эпизод', 'шли', 'чувствую', 'чувствовал', 'чувствительные', 'чистейшей', 'число', 'черный', 'человек

In [86]:
evaluate(data['keywords'], rake_keywords)

Precision -  0.02
Recall -  0.06
F1 -  0.03
Jaccard -  0.01


Попытка дать нормализованные тексты из списков только ухудшила результат.

In [96]:
def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS == 'NOUN' or word.tag.POS == 'ADJF' or word.tag.POS == 'ADJS']

    return words

In [97]:
data['content_norm'] = data['content'].apply(normalize)

In [102]:
# Попытка 2
r = Rake(language="ru", stopwords=list(stop_words), punctuations=punctuations, min_length=1, max_length=1)

rake_keywords = []
for text in data['content_norm']:
    r.extract_keywords_from_text(' '.join(text))
    keywords = r.get_ranked_phrases()
    #c=r.get_ranked_phrases_with_scores()
    rake_keywords.append(keywords[:20])
print(rake_keywords)


[['момент'], ['скамейка', 'образ', 'манер', 'красота', 'кошка', 'занавеска', 'деньга'], [], ['сезон', 'неудачный', 'кое', 'гран', 'гонка'], ['самый', 'иной', 'есенин', 'выходка'], ['выпускник'], ['свадьба', 'книга', 'задача', 'дыхание'], ['рияда', 'аль'], ['элемент', 'нейробиолог', 'креативный', 'биокомпьютер', 'античный'], ['россиянин'], ['полный', 'интересный', 'довлат', 'автор'], ['сговорчивый', 'русский', 'мальчик', 'злато', 'больница', 'берухо'], ['хипстерский', 'параграф', 'незнающий', 'глаз', 'ганзейский', 'арт'], ['терпимость', 'последствие'], ['телевизор', 'реальный', 'макс', 'канал', 'госдума', 'главный', 'выпуск'], ['обсе', 'британия'], ['осадка', 'зелёный', 'житель'], ['сентябрь', 'последний'], ['финал', 'снятой', 'рука', 'отношение', 'компания', 'город', 'агония'], ['случай'], ['страна', 'премьер', 'нервно', 'би'], ['поза', 'основное', 'влияние'], ['март'], ['учёт', 'ближний'], [], ['попытка', 'половина', 'очередь', 'голосование'], ['риск', 'работа', 'проблема', 'новизна',

In [104]:
evaluate(data['keywords'], rake_keywords)

Precision -  0.04
Recall -  0.02
F1 -  0.02
Jaccard -  0.01
